In [ ]:
@alice = key :new
@bob = key :new
get_height

In [ ]:
extend_chain to: @alice, num_blocks: 101

In [ ]:
@alice_coinbase = spendable_coinbase_for @alice

## Runes

### Minting

In a single transaction a rune is minted and transferred to Alice
1. We create 100 runes with symbol 'PINK'
2. We transfer runs to first output - essentially making Alice own all the PINK runes

### Serialization

We don't follow the exact serialization algorithm described in the links below. Instead, we just do some etching, minting and transfers to communicate the essence of Runes. The serialization semantics described in the ordinals repository help push multiple, etchings, minting and transfers in a single OP_RETURN.

References to serialization:

https://github.com/ordinals/ord/blob/master/docs/src/runes/specification.md

https://docs.ordinals.com/runes/specification.html?highlight=base-26#deciphering

# Etching a Rune

Runes come into existence by being ethced. There are a few advanced things we can do, but here we just focus on the divisibilty and the name. We use 'AB' as the name and provide a divisibility of eight.

In [ ]:
@marker = Bitcoin::Opcodes::OP_13 # marker for runestone
@etch = [8, 27]  # divisibility: 8, rune 27 is 'AB' in modified base-26

In [ ]:
@alice_etches_runes = transaction inputs: [ {tx: @alice_coinbase, vout: 0, script_sig: 'sig:wpkh(@alice)' }],
                                outputs: [ 
                                    {amount: 49.998.sats, script: 'OP_DUP OP_HASH160 hash160(@alice) OP_EQUALVERIFY OP_CHECKSIG'},
                                    {amount: 0, script: 'OP_RETURN @etch'}
                                ]

In [ ]:
broadcast @alice_etches_runes
confirm transaction: @alice_etches_runes

# Minting Runes

Once a rune has been "etched", we can start minting it. In practice the transaction that etches runes will also mint or premine them - both supported by the runes protocol. Here we let anyone mint some 'AB' runes. 

## Rune ID
First We need a rune id, which is blockhash and txid pair - so we get that next.

In [ ]:
@blockhash = get_block_confirmed_at transaction: @alice_etches_runes
@rune_id = "#{@blockhash}:#{@alice_etches_runes.txid}"

# Minting transaction

Alice can now mint runes. In practice, ethcing, minting, transfer can all happen in the same transaction. We avoid doing so here to make it easier to describe the protocol, and also to avoid writing the runes serde protocol.

In [ ]:
assert_output_is_spent transaction: @alice_coinbase, vout: 0

In [ ]:
@alice_mints_runes = transaction inputs: [ {tx: @alice_coinbase, vout: 0, script_sig: 'sig:wpkh(@alice)' } ],
                                 outputs: [ {amount: 49.997.sats, script: 'OP_DUP OP_HASH160 hash160(@bob) OP_EQUALVERIFY OP_CHECKSIG OP_RETURN} ]